# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import os
import requests
from bs4 import BeautifulSoup

# Global Variables

In [22]:
BASE_URL = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos-e-pronunciamentos'
DATA_DIR = '/content/drive/MyDrive/DS_PROJECTS/pronouncements'

# Connecting to the URL

In [4]:
response = requests.get(BASE_URL)

In [5]:
response.text

'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="pt-br" xml:lang="pt-br">\n\n<head>\n    <meta charset="utf-8" />\n  <!-- Google Tag Manager -->\n  <script type="text">(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n  new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n  j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n  \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n  })(window,document,\'script\',\'dataLayer\',\'GTM-NHZSKTD\');</script>\n  <!-- End Google Tag Manager -->\n\n        <script type="application/ld+json">\n        {\n            "@context": "https://schema.org",\n            "@type": "Organization",\n            "url": "https://www.gov.br/planalto",\n            "logo": "https://www.gov.br/planalto/logo.png"\n        }</script>\n        <script type="application/ld+json">\n        {\n            "@context": "https://schema.org",\n            "@type":

# Finding metadata from each speech

In [6]:
bs = BeautifulSoup(response.text)

In [7]:
articles = bs.find_all('article')

In [8]:
articles[0]

<article class="entry">
<header>
<span class="summary">
<a class="state-published url" href="https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos-e-pronunciamentos/2023/discurso-do-presidente-da-republica-luiz-inacio-lula-da-silva-em-cerimonia-alusiva-ao-dia-internacional-da-luta-pela-eliminacao-da-discriminacao-racial" title="Document">DISCURSO do presidente da República, Luiz Inácio Lula da Silva, em cerimônia alusiva ao Dia Internacional da Luta pela Eliminação da Discriminação Racial</a>
</span>
<span class="documentByLine">
                        
                        
                          
                          

                          
                            —
                            última modificação
                            21/03/2023 17h34
                          

                          
                            
                          
                        
                      </span>
</header>
<p class="description d

In [9]:
print(f'{len(articles)} pronouncements')

30 pronouncements


In [10]:
article = articles[0]
url = article.find(class_='state-published url')
tags = article.find_all(title='Document')
datetime = article.find_all(class_='documentByLine')

In [11]:
url['href']

'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos-e-pronunciamentos/2023/discurso-do-presidente-da-republica-luiz-inacio-lula-da-silva-em-cerimonia-alusiva-ao-dia-internacional-da-luta-pela-eliminacao-da-discriminacao-racial'

In [12]:
tags = [t.text for t in tags]

In [13]:
tags 

['DISCURSO do presidente da República, Luiz Inácio Lula da Silva, em cerimônia alusiva ao Dia Internacional da Luta pela Eliminação da Discriminação Racial']

In [14]:
datetime

[<span class="documentByLine">
                         
                         
                           
                           
 
                           
                             —
                             última modificação
                             21/03/2023 17h34
                           
 
                           
                             
                           
                         
                       </span>]

In [15]:
date = datetime[0].text.strip().split()[3]
time = datetime[0].text.strip().split()[4]

In [16]:
date, time

('21/03/2023', '17h34')

In [17]:
def canonical_date(date_str):
  '''
  Input: string data
  Output: string canonical date -> YYYYMMDD
  '''
  return ''.join(reversed(date_str.split('/')))

In [18]:
canonical_date(date)

'20230321'

In [19]:
class Pronouncements:
  def __init__(self, bs_article=None):

    if bs_article is not None:
      self.url = bs_article.find(class_='state-published url')['href']
      self.tags = [t.text for t in bs_article.find_all(title='Document')]

      datetime = bs_article.find_all(class_='documentByLine')
      self.date = canonical_date(datetime[0].text.strip().split()[3])
      self.time = datetime[0].text.strip().split()[4]

      self.text = self._extract_text()

  def _extract_text(self):
    r = requests.get(self.url)
    bs = BeautifulSoup(r.text)
    return bs.find(id='parent-fieldname-text').text.strip()

  def save2json(pronouncement, save_dir, name=None):
    if name is None:
      name = f'{pronouncement.date}-{pronouncement.time}.json'
    path = os.path.join(save_dir, name)
    with open(path, 'w') as jsonfile:
      json.dump(pronouncement.__dict__, jsonfile)
  
  def load_from_json(filepath):
    with open(filepath, 'r') as jsonfile:
      data = json.load(jsonfile)
      p = Pronouncements()
      p.date = data['date']
      p.time = data['time']
      p.tags = data['tags']
      p.url  = data['url']
      p.text = data['text']
      return p

In [20]:
pronouncements = [Pronouncements(art) for art in articles]

In [23]:
for p in pronouncements:
  Pronouncements.save2json(p, DATA_DIR)

In [25]:
filenames = os.listdir(DATA_DIR)

In [26]:
pronounc = []
for name in filenames:
  path = os.path.join(DATA_DIR, name)
  pronounc.append(Pronouncements.load_from_json(path))
len(pronounc)

30

In [36]:
for i in range(len(pronounc)):
  print(pronounc[i].date, pronounc[i].time)
  print(pronounc[i].text)
  print('==============================')

20230321 17h34
Quero saudar a ministra Anielle Franco e toda a equipe pela excelência do trabalho apresentado. Vocês são o testemunho vivo da importância de haver cada vez mais pessoas negras em postos-chave do governo, formulando políticas e tomando decisões em nome da maioria da população.
O que a ministra Anielle e este governo vão realizar nos próximos quatro anos será o desdobramento lógico de uma caminhada que começou há duas décadas, quando criamos a Secretaria Especial de Políticas de Promoção da Igualdade Racial, em 21 de março de 2003.
A criação da Seppir foi precedida pela sanção – ainda nos primeiros dias de meu primeiro mandato – da Lei nº 10.639, proposta pela educadora Esther Grossi. A lei tornou obrigatório o ensino da história e cultura afro-brasileira nas escolas de nível fundamental e médio.
É estudando a contribuição cultural, econômica e política dos povos africanos e seus descendentes, que crianças e jovens de todas as etnias aprendem a reconhecer e a admirar esse